In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset

In [2]:
raw_df=pd.read_csv("Bank Customer Churn Prediction.csv",index_col=0)
raw_df.drop_duplicates()
display(raw_df)

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
customer_id,,,,,,,,,,,
15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
15606229,771,France,Male,39,5,0.00,2,1,0,96270.64,0
15569892,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
15584532,709,France,Female,36,7,0.00,1,0,1,42085.58,1


In [3]:
man_df=raw_df.copy().loc[raw_df["gender"]=="Male"].loc[:,['gender','churn']]
woman_df=raw_df.copy().loc[raw_df["gender"]!="Male"].loc[:,['gender','churn']]

def get_percentage(value):
    return round(value,4)*100

print(f"The privillege gorups take up {get_percentage(man_df.shape[0]/raw_df.shape[0])}% samples")
print(f"The unprivillege gorups take up {get_percentage(woman_df.shape[0]/raw_df.shape[0])}% samples")

man_label=man_df["churn"].value_counts()
print(f"The privillege groups have {get_percentage(man_label[1]/man_df.shape[0])}% positive label")
print(f"The privillege groups have {get_percentage(man_label[0]/man_df.shape[0])}% negative lablel")

women_label=woman_df["churn"].value_counts()
print(f"The unprivillege groups have {get_percentage(women_label[1]/woman_df.shape[0])}% positive label")
print(f"The unprivillege groups have {get_percentage(women_label[0]/woman_df.shape[0])}% negative lablel")

The privillege gorups take up 54.56999999999999% samples
The unprivillege gorups take up 45.43% samples
The privillege groups have 16.46% positive label
The privillege groups have 83.54% negative lablel
The unprivillege groups have 25.069999999999997% positive label
The unprivillege groups have 74.92999999999999% negative lablel


In [4]:
def min_max_normalisation(series):
    return (series - series.min()) / (series.max() - series.min())

df=raw_df.copy(deep=True)
#normalize PreviousSalary feature
df["balance"]=min_max_normalisation(df["balance"])
df["estimated_salary"]=min_max_normalisation(df["estimated_salary"])
df["credit_score"]=min_max_normalisation(df["credit_score"])
df["age"]=min_max_normalisation(df["age"])
df=df.reset_index().drop(["customer_id"],axis=1)

display(df)

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,0.538,France,Female,0.324324,2,0.000000,1,1,1,0.506735,1
1,0.516,Spain,Female,0.310811,1,0.334031,1,0,1,0.562709,0
2,0.304,France,Female,0.324324,8,0.636357,3,1,0,0.569654,1
3,0.698,France,Female,0.283784,1,0.000000,2,0,0,0.469120,0
4,1.000,Spain,Female,0.337838,2,0.500246,1,1,1,0.395400,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,0.842,France,Male,0.283784,5,0.000000,2,1,0,0.481341,0
9996,0.332,France,Male,0.229730,10,0.228657,1,1,1,0.508490,0
9997,0.718,France,Female,0.243243,7,0.000000,1,0,1,0.210390,1
9998,0.844,Germany,Male,0.324324,3,0.299226,2,1,0,0.464429,1


In [5]:
folds_index=np.array_split(df.sample(frac=1).index,5)
raw_dataset=StandardDataset(df=df,label_name="churn",favorable_classes=[1],scores_name="",protected_attribute_names=["gender"],
                                    privileged_classes=[["Male"]],categorical_features=['country'])

In [6]:
Cross_Validation_Column=["Cross Validate 1","Cross Validate 2","Cross Validate 3","Cross Validate 4","Cross Validate 5","Average"]
Accuracy_data=pd.DataFrame(columns=Cross_Validation_Column)
Precision_data=pd.DataFrame(columns=Cross_Validation_Column)
Recall_data=pd.DataFrame(columns=Cross_Validation_Column)
DisparateImpact_data=pd.DataFrame(columns=Cross_Validation_Column)
DisparateParity_data=pd.DataFrame(columns=Cross_Validation_Column)
EqualOpportunity_data=pd.DataFrame(columns=Cross_Validation_Column)
EqualizedOdds_data=pd.DataFrame(columns=Cross_Validation_Column)
#Read progress from local file if something happend that the jupyter kernel needs to be restarted (like fix bug from source library and reloaded it)
#merics_data=pd.read_csv("Benchmark_data.csv",index_col=0)

privileged_groups = [{"gender":1}]
unprivileged_groups = [{"gender":0}]

In [7]:
def cross_validation(row_name,train_test_process):
    global Accuracy_data
    global Precision_data
    global Recall_data
    global DisparateImpact_data
    global DisparateParity_data
    global EqualOpportunity_data
    global EqualizedOdds_data
    accuracy_row=[]
    precision_row=[]
    recall_row=[]
    disparateImpcat_row=[]
    disparateParity_row=[]
    EqualOpportunity_row=[]
    EqualizedOdss_row=[]
    for index in range(5):
        test_fold=folds_index[index]
        train_index=pd.Index([])
        for f in folds_index:
            if f is not test_fold:
                train_index=train_index.append(f)
        train_set=raw_dataset.subset(train_index)
        test_set=raw_dataset.subset(test_fold)
        test_pred=train_test_process(train_set,test_set)
        metric=ClassificationMetric(test_set,test_pred,unprivileged_groups=unprivileged_groups,
                                                privileged_groups=privileged_groups)
        #Record Result
        accuracy_row.append(metric.accuracy())
        precision_row.append(metric.precision())
        recall_row.append(metric.recall())
        disparateImpcat_row.append(metric.disparate_impact())
        disparateParity_row.append(metric.statistical_parity_difference())
        EqualOpportunity_row.append(metric.true_positive_rate_difference())
        EqualizedOdss_row.append(metric.false_positive_rate_difference())
    accuracy_row.append(np.average(accuracy_row))
    precision_row.append(np.average(precision_row))
    recall_row.append(np.average(recall_row))
    disparateImpcat_row.append(np.average(disparateImpcat_row))
    disparateParity_row.append(np.average(disparateParity_row))
    EqualOpportunity_row.append(np.average(EqualOpportunity_row))
    EqualizedOdss_row.append(np.average(EqualizedOdss_row))
    #Record Cross Validation
    Accuracy_data=pd.concat([Accuracy_data,pd.DataFrame([accuracy_row],index=[row_name],columns=Cross_Validation_Column)])
    Accuracy_data.columns.name="Accuracy"
    display(Accuracy_data)
    Precision_data=pd.concat([Precision_data,pd.DataFrame([precision_row],index=[row_name],columns=Cross_Validation_Column)])
    Precision_data.columns.name="Precision"
    display(Precision_data)
    Recall_data=pd.concat([Recall_data,pd.DataFrame([recall_row],index=[row_name],columns=Cross_Validation_Column)])
    Recall_data.columns.name="Recall"
    display(Recall_data)
    DisparateImpact_data=pd.concat([DisparateImpact_data,pd.DataFrame([disparateImpcat_row],index=[row_name],columns=Cross_Validation_Column)])
    DisparateImpact_data.columns.name="Disparate Impact"
    display(DisparateImpact_data)
    DisparateParity_data=pd.concat([DisparateParity_data,pd.DataFrame([disparateParity_row],index=[row_name],columns=Cross_Validation_Column)])
    DisparateParity_data.columns.name="Disparate Parity"
    display(DisparateParity_data)
    EqualOpportunity_data=pd.concat([EqualOpportunity_data,pd.DataFrame([EqualOpportunity_row],index=[row_name],columns=Cross_Validation_Column)])
    EqualOpportunity_data.columns.name="Equal Opportuniy"
    display(EqualOpportunity_data)
    EqualizedOdds_data=pd.concat([EqualizedOdds_data,pd.DataFrame([EqualizedOdss_row],index=[row_name],columns=Cross_Validation_Column)])
    EqualizedOdds_data.columns.name="Equalized Odds"
    display(EqualizedOdds_data)

In [8]:
#Train RandomForest as Baseline model on original Datasets
def baseline_CV(train_set, test_set):
    #Training
    baseline=RandomForestClassifier()
    baseline.fit(train_set.features,train_set.labels.ravel())
    #Testing
    pred=baseline.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("Baseline",baseline_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.8565,0.8665,0.874,0.8575,0.8565,0.8622


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.712446,0.747826,0.812245,0.740458,0.784983,0.759592


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.430052,0.451444,0.491358,0.472019,0.506608,0.470296


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.726181,1.271284,1.576324,2.064563,1.613777,1.650426


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.064132,0.027666,0.056061,0.093348,0.070543,0.06235


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.130262,-0.102078,0.034698,0.125424,-0.037054,0.03025


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.024258,0.003925,0.00923,0.036828,0.044597,0.023768


In [9]:
#Zhange et al. implementation
import tensorflow.compat.v1 as tf
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
def AdversarialDebiase_CV(train_set,test_set):
    #Training
    tf.disable_eager_execution()
    tf.reset_default_graph()
    with tf.Session() as sess:
        debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                                unprivileged_groups = unprivileged_groups,
                                scope_name='debiased_classifier',
                                debias=True,
                                sess=sess)
        debiased_model.fit(train_set)
    #Testing
        test_pred=debiased_model.predict(test_set)
    return test_pred

cross_validation("AdversarialDebiase",AdversarialDebiase_CV)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 1.003834; batch adversarial loss: 0.722480
epoch 1; iter: 0; batch classifier loss: 0.474092; batch adversarial loss: 0.699300
epoch 2; iter: 0; batch classifier loss: 0.587638; batch adversarial loss: 0.694593
epoch 3; iter: 0; batch classifier loss: 0.515344; batch adversarial loss: 0.702464
epoch 4; iter: 0; batch classifier loss: 0.404858; batch adversarial loss: 0.696179
epoch 5; iter: 0; batch classifier loss: 0.462121; batch adversarial loss: 0.684796
epoch 6; iter: 0; batch classifier loss: 0.443965; batch adversarial loss: 0.685906
epoch 7; iter: 0; batch classifier loss: 0.417797; batch adversarial loss: 0.680758
epoch 8; iter: 0; batch classifier loss: 0.450916; batch adversarial loss: 0.687363
epoch 9; iter: 0; batch classifier loss: 0.384119; batch adversarial loss: 0.695266
epoch 10; iter: 0; batch classifier loss: 0.417743; batch adversarial loss: 0.697216
epoch 11; iter: 0; batch classifier loss: 0.429177; batch adversarial loss:

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.8565,0.8665,0.874,0.8575,0.8565,0.8622
AdversarialDebiase,0.8475,0.8680,0.854,0.8540,0.8490,0.8545


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.712446,0.747826,0.812245,0.740458,0.784983,0.759592
AdversarialDebiase,0.668050,0.762332,0.760369,0.828729,0.790076,0.761911


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.430052,0.451444,0.491358,0.472019,0.506608,0.470296
AdversarialDebiase,0.417098,0.446194,0.407407,0.364964,0.455947,0.418322


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.726181,1.271284,1.576324,2.064563,1.613777,1.650426
AdversarialDebiase,1.070873,1.602470,1.512027,1.276063,1.312409,1.354768


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.064132,0.027666,0.056061,0.093348,0.070543,0.062350
AdversarialDebiase,0.008282,0.052339,0.045152,0.022147,0.035906,0.032765


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.130262,-0.102078,0.034698,0.125424,-0.037054,0.030250
AdversarialDebiase,0.021585,-0.000779,0.055314,-0.031162,-0.030675,0.002856


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.024258,0.003925,0.009230,0.036828,0.044597,0.023768
AdversarialDebiase,-0.017635,0.011934,0.000151,-0.003841,0.002899,-0.001298


In [10]:
#Agarwal et al., 2018, Agarwal et al., 2019 Implementation
from aif360.algorithms.inprocessing import GridSearchReduction
def GridSearchReduction_EO_CV(train_set,test_set):
    #Training
    gsr_eo_model=GridSearchReduction(estimator=RandomForestClassifier(),constraints="EqualizedOdds",prot_attr="gender")
    gsr_eo_model.fit(train_set)
    #Testing
    test_pred=gsr_eo_model.predict(test_set)
    return test_pred

cross_validation("GridSearchReduction(EO)",GridSearchReduction_EO_CV)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\prepro

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.8565,0.8665,0.874,0.8575,0.8565,0.8622
AdversarialDebiase,0.8475,0.8680,0.854,0.8540,0.8490,0.8545
GridSearchReduction(EO),0.8605,0.8675,0.872,0.8580,0.8570,0.8630


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.712446,0.747826,0.812245,0.740458,0.784983,0.759592
AdversarialDebiase,0.668050,0.762332,0.760369,0.828729,0.790076,0.761911
GridSearchReduction(EO),0.729614,0.741667,0.796813,0.763485,0.795775,0.765471


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.430052,0.451444,0.491358,0.472019,0.506608,0.470296
AdversarialDebiase,0.417098,0.446194,0.407407,0.364964,0.455947,0.418322
GridSearchReduction(EO),0.440415,0.467192,0.493827,0.447689,0.497797,0.469384


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.726181,1.271284,1.576324,2.064563,1.613777,1.650426
AdversarialDebiase,1.070873,1.602470,1.512027,1.276063,1.312409,1.354768
GridSearchReduction(EO),1.377793,1.144313,1.399810,1.547707,1.287931,1.351511


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.064132,0.027666,0.056061,0.093348,0.070543,0.062350
AdversarialDebiase,0.008282,0.052339,0.045152,0.022147,0.035906,0.032765
GridSearchReduction(EO),0.037746,0.016216,0.042525,0.052625,0.036219,0.037066


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.130262,-0.102078,0.034698,0.125424,-0.037054,0.030250
AdversarialDebiase,0.021585,-0.000779,0.055314,-0.031162,-0.030675,0.002856
GridSearchReduction(EO),0.078340,-0.142078,0.009033,0.023341,-0.061900,-0.018653


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.024258,0.003925,0.009230,0.036828,0.044597,0.023768
AdversarialDebiase,-0.017635,0.011934,0.000151,-0.003841,0.002899,-0.001298
GridSearchReduction(EO),0.003201,-0.002984,-0.001334,0.013252,0.007557,0.003938


In [11]:
#Agarwal et al., 2018, Agarwal et al., 2019 Implementation

def GridSearchReduction_DP_CV(train_set,test_set):
    #Training
    gsr_dp_model=GridSearchReduction(estimator=RandomForestClassifier(),constraints="DemographicParity",prot_attr="gender")
    gsr_dp_model.fit(train_set)
    #Testing
    test_pred=gsr_dp_model.predict(test_set)
    return test_pred

cross_validation("GridSearchReduction(DP)",GridSearchReduction_DP_CV)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\prepro

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.8565,0.8665,0.874,0.8575,0.8565,0.8622
AdversarialDebiase,0.8475,0.8680,0.854,0.8540,0.8490,0.8545
GridSearchReduction(EO),0.8605,0.8675,0.872,0.8580,0.8570,0.8630
GridSearchReduction(DP),0.8625,0.8640,0.873,0.8550,0.8580,0.8625


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.712446,0.747826,0.812245,0.740458,0.784983,0.759592
AdversarialDebiase,0.668050,0.762332,0.760369,0.828729,0.790076,0.761911
GridSearchReduction(EO),0.729614,0.741667,0.796813,0.763485,0.795775,0.765471
GridSearchReduction(DP),0.744493,0.746606,0.803213,0.746939,0.797203,0.767691


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.430052,0.451444,0.491358,0.472019,0.506608,0.470296
AdversarialDebiase,0.417098,0.446194,0.407407,0.364964,0.455947,0.418322
GridSearchReduction(EO),0.440415,0.467192,0.493827,0.447689,0.497797,0.469384
GridSearchReduction(DP),0.437824,0.433071,0.493827,0.445255,0.502203,0.462436


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.726181,1.271284,1.576324,2.064563,1.613777,1.650426
AdversarialDebiase,1.070873,1.602470,1.512027,1.276063,1.312409,1.354768
GridSearchReduction(EO),1.377793,1.144313,1.399810,1.547707,1.287931,1.351511
GridSearchReduction(DP),1.380614,1.135629,1.335201,1.465358,1.287550,1.320870


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.064132,0.027666,0.056061,0.093348,0.070543,0.062350
AdversarialDebiase,0.008282,0.052339,0.045152,0.022147,0.035906,0.032765
GridSearchReduction(EO),0.037746,0.016216,0.042525,0.052625,0.036219,0.037066
GridSearchReduction(DP),0.037009,0.014088,0.036263,0.046883,0.036432,0.034135


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.130262,-0.102078,0.034698,0.125424,-0.037054,0.030250
AdversarialDebiase,0.021585,-0.000779,0.055314,-0.031162,-0.030675,0.002856
GridSearchReduction(EO),0.078340,-0.142078,0.009033,0.023341,-0.061900,-0.018653
GridSearchReduction(DP),0.062822,-0.132381,-0.000990,0.009153,-0.053932,-0.023066


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.024258,0.003925,0.009230,0.036828,0.044597,0.023768
AdversarialDebiase,-0.017635,0.011934,0.000151,-0.003841,0.002899,-0.001298
GridSearchReduction(EO),0.003201,-0.002984,-0.001334,0.013252,0.007557,0.003938
GridSearchReduction(DP),0.006047,-0.003909,-0.006877,0.010238,0.004896,0.002079


In [12]:
#Kamiran and Calders, 2012 implementation
from aif360.algorithms.preprocessing import Reweighing
def Reweight_CV(train_set,test_set):
    #Training
    reweight=Reweighing(privileged_groups=privileged_groups,unprivileged_groups=unprivileged_groups)
    reweight.fit(train_set)
    reweight_set=reweight.transform(train_set.copy(deepcopy=True))
    reweight_test = reweight.transform(test_set.copy(deepcopy=True))

    reweight_model=RandomForestClassifier()
    reweight_model.fit(reweight_set.features,reweight_set.labels.ravel(),sample_weight=train_set.instance_weights)
    #Testing
    pred=reweight_model.predict(reweight_test.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("Reweight",Reweight_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.8565,0.8665,0.874,0.8575,0.8565,0.8622
AdversarialDebiase,0.8475,0.8680,0.854,0.8540,0.8490,0.8545
GridSearchReduction(EO),0.8605,0.8675,0.872,0.8580,0.8570,0.8630
GridSearchReduction(DP),0.8625,0.8640,0.873,0.8550,0.8580,0.8625
Reweight,0.8600,0.8640,0.870,0.8570,0.8570,0.8616


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.712446,0.747826,0.812245,0.740458,0.784983,0.759592
AdversarialDebiase,0.668050,0.762332,0.760369,0.828729,0.790076,0.761911
GridSearchReduction(EO),0.729614,0.741667,0.796813,0.763485,0.795775,0.765471
GridSearchReduction(DP),0.744493,0.746606,0.803213,0.746939,0.797203,0.767691
Reweight,0.732456,0.733906,0.793522,0.743191,0.781879,0.756991


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.430052,0.451444,0.491358,0.472019,0.506608,0.470296
AdversarialDebiase,0.417098,0.446194,0.407407,0.364964,0.455947,0.418322
GridSearchReduction(EO),0.440415,0.467192,0.493827,0.447689,0.497797,0.469384
GridSearchReduction(DP),0.437824,0.433071,0.493827,0.445255,0.502203,0.462436
Reweight,0.432642,0.448819,0.483951,0.464720,0.513216,0.468670


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.726181,1.271284,1.576324,2.064563,1.613777,1.650426
AdversarialDebiase,1.070873,1.602470,1.512027,1.276063,1.312409,1.354768
GridSearchReduction(EO),1.377793,1.144313,1.399810,1.547707,1.287931,1.351511
GridSearchReduction(DP),1.380614,1.135629,1.335201,1.465358,1.287550,1.320870
Reweight,1.817999,1.302545,1.625786,1.905439,1.450924,1.620539


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.064132,0.027666,0.056061,0.093348,0.070543,0.062350
AdversarialDebiase,0.008282,0.052339,0.045152,0.022147,0.035906,0.032765
GridSearchReduction(EO),0.037746,0.016216,0.042525,0.052625,0.036219,0.037066
GridSearchReduction(DP),0.037009,0.014088,0.036263,0.046883,0.036432,0.034135
Reweight,0.068592,0.030854,0.060303,0.081929,0.055906,0.059517


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.130262,-0.102078,0.034698,0.125424,-0.037054,0.030250
AdversarialDebiase,0.021585,-0.000779,0.055314,-0.031162,-0.030675,0.002856
GridSearchReduction(EO),0.078340,-0.142078,0.009033,0.023341,-0.061900,-0.018653
GridSearchReduction(DP),0.062822,-0.132381,-0.000990,0.009153,-0.053932,-0.023066
Reweight,0.156143,-0.084416,0.041528,0.092809,-0.060742,0.029064


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.024258,0.003925,0.009230,0.036828,0.044597,0.023768
AdversarialDebiase,-0.017635,0.011934,0.000151,-0.003841,0.002899,-0.001298
GridSearchReduction(EO),0.003201,-0.002984,-0.001334,0.013252,0.007557,0.003938
GridSearchReduction(DP),0.006047,-0.003909,-0.006877,0.010238,0.004896,0.002079
Reweight,0.023108,0.004541,0.014089,0.031378,0.031734,0.020970


In [13]:
#Feldman et al., 2015 implementation
from aif360.algorithms.preprocessing import DisparateImpactRemover

def DisparateImpactRemover_05_CV(train_set,test_set):
    #Training
    di = DisparateImpactRemover(repair_level=0.5,sensitive_attribute="gender")
    train_di = di.fit_transform(train_set.copy(deepcopy=True))
    test_di = di.fit_transform(test_set.copy(deepcopy=True))

    di_model=RandomForestClassifier()
    di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)
    #Testing
    pred=di_model.predict(test_di.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("DisparateImpactRemover(0.5)",DisparateImpactRemover_05_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.8565,0.8665,0.874,0.8575,0.8565,0.8622
AdversarialDebiase,0.8475,0.8680,0.854,0.8540,0.8490,0.8545
GridSearchReduction(EO),0.8605,0.8675,0.872,0.8580,0.8570,0.8630
GridSearchReduction(DP),0.8625,0.8640,0.873,0.8550,0.8580,0.8625
Reweight,0.8600,0.8640,0.870,0.8570,0.8570,0.8616
DisparateImpactRemover(0.5),0.8575,0.8725,0.873,0.8550,0.8555,0.8627


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.712446,0.747826,0.812245,0.740458,0.784983,0.759592
AdversarialDebiase,0.668050,0.762332,0.760369,0.828729,0.790076,0.761911
GridSearchReduction(EO),0.729614,0.741667,0.796813,0.763485,0.795775,0.765471
GridSearchReduction(DP),0.744493,0.746606,0.803213,0.746939,0.797203,0.767691
Reweight,0.732456,0.733906,0.793522,0.743191,0.781879,0.756991
DisparateImpactRemover(0.5),0.716738,0.766949,0.791506,0.741036,0.791519,0.761550


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.430052,0.451444,0.491358,0.472019,0.506608,0.470296
AdversarialDebiase,0.417098,0.446194,0.407407,0.364964,0.455947,0.418322
GridSearchReduction(EO),0.440415,0.467192,0.493827,0.447689,0.497797,0.469384
GridSearchReduction(DP),0.437824,0.433071,0.493827,0.445255,0.502203,0.462436
Reweight,0.432642,0.448819,0.483951,0.464720,0.513216,0.468670
DisparateImpactRemover(0.5),0.432642,0.475066,0.506173,0.452555,0.493392,0.471966


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.726181,1.271284,1.576324,2.064563,1.613777,1.650426
AdversarialDebiase,1.070873,1.602470,1.512027,1.276063,1.312409,1.354768
GridSearchReduction(EO),1.377793,1.144313,1.399810,1.547707,1.287931,1.351511
GridSearchReduction(DP),1.380614,1.135629,1.335201,1.465358,1.287550,1.320870
Reweight,1.817999,1.302545,1.625786,1.905439,1.450924,1.620539
DisparateImpactRemover(0.5),1.696166,1.333806,1.604167,1.996439,1.516562,1.629428


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.064132,0.027666,0.056061,0.093348,0.070543,0.062350
AdversarialDebiase,0.008282,0.052339,0.045152,0.022147,0.035906,0.032765
GridSearchReduction(EO),0.037746,0.016216,0.042525,0.052625,0.036219,0.037066
GridSearchReduction(DP),0.037009,0.014088,0.036263,0.046883,0.036432,0.034135
Reweight,0.068592,0.030854,0.060303,0.081929,0.055906,0.059517
DisparateImpactRemover(0.5),0.062102,0.034043,0.061515,0.085515,0.059370,0.060509


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.130262,-0.102078,0.034698,0.125424,-0.037054,0.030250
AdversarialDebiase,0.021585,-0.000779,0.055314,-0.031162,-0.030675,0.002856
GridSearchReduction(EO),0.078340,-0.142078,0.009033,0.023341,-0.061900,-0.018653
GridSearchReduction(DP),0.062822,-0.132381,-0.000990,0.009153,-0.053932,-0.023066
Reweight,0.156143,-0.084416,0.041528,0.092809,-0.060742,0.029064
DisparateImpactRemover(0.5),0.135417,-0.096104,0.051106,0.121380,-0.043138,0.033732


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.024258,0.003925,0.009230,0.036828,0.044597,0.023768
AdversarialDebiase,-0.017635,0.011934,0.000151,-0.003841,0.002899,-0.001298
GridSearchReduction(EO),0.003201,-0.002984,-0.001334,0.013252,0.007557,0.003938
GridSearchReduction(DP),0.006047,-0.003909,-0.006877,0.010238,0.004896,0.002079
Reweight,0.023108,0.004541,0.014089,0.031378,0.031734,0.020970
DisparateImpactRemover(0.5),0.020262,0.007225,0.010835,0.029933,0.033063,0.020264


In [14]:
#Feldman et al., 2015 implementation
def DisparateImpactRemover_07_CV(train_set,test_set):
    #Training
    di = DisparateImpactRemover(repair_level=0.7,sensitive_attribute="gender")
    train_di = di.fit_transform(train_set.copy(deepcopy=True))
    test_di = di.fit_transform(test_set.copy(deepcopy=True))

    di_model=RandomForestClassifier()
    di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)
    #Testing
    pred=di_model.predict(test_di.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("DisparateImpactRemover(0.7)",DisparateImpactRemover_07_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.8565,0.8665,0.874,0.8575,0.8565,0.8622
AdversarialDebiase,0.8475,0.8680,0.854,0.8540,0.8490,0.8545
GridSearchReduction(EO),0.8605,0.8675,0.872,0.8580,0.8570,0.8630
GridSearchReduction(DP),0.8625,0.8640,0.873,0.8550,0.8580,0.8625
Reweight,0.8600,0.8640,0.870,0.8570,0.8570,0.8616
DisparateImpactRemover(0.5),0.8575,0.8725,0.873,0.8550,0.8555,0.8627
DisparateImpactRemover(0.7),0.8590,0.8710,0.875,0.8515,0.8550,0.8623


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.712446,0.747826,0.812245,0.740458,0.784983,0.759592
AdversarialDebiase,0.668050,0.762332,0.760369,0.828729,0.790076,0.761911
GridSearchReduction(EO),0.729614,0.741667,0.796813,0.763485,0.795775,0.765471
GridSearchReduction(DP),0.744493,0.746606,0.803213,0.746939,0.797203,0.767691
Reweight,0.732456,0.733906,0.793522,0.743191,0.781879,0.756991
DisparateImpactRemover(0.5),0.716738,0.766949,0.791506,0.741036,0.791519,0.761550
DisparateImpactRemover(0.7),0.726087,0.761702,0.813765,0.729839,0.788732,0.764025


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.430052,0.451444,0.491358,0.472019,0.506608,0.470296
AdversarialDebiase,0.417098,0.446194,0.407407,0.364964,0.455947,0.418322
GridSearchReduction(EO),0.440415,0.467192,0.493827,0.447689,0.497797,0.469384
GridSearchReduction(DP),0.437824,0.433071,0.493827,0.445255,0.502203,0.462436
Reweight,0.432642,0.448819,0.483951,0.464720,0.513216,0.468670
DisparateImpactRemover(0.5),0.432642,0.475066,0.506173,0.452555,0.493392,0.471966
DisparateImpactRemover(0.7),0.432642,0.469816,0.496296,0.440389,0.493392,0.466507


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.726181,1.271284,1.576324,2.064563,1.613777,1.650426
AdversarialDebiase,1.070873,1.602470,1.512027,1.276063,1.312409,1.354768
GridSearchReduction(EO),1.377793,1.144313,1.399810,1.547707,1.287931,1.351511
GridSearchReduction(DP),1.380614,1.135629,1.335201,1.465358,1.287550,1.320870
Reweight,1.817999,1.302545,1.625786,1.905439,1.450924,1.620539
DisparateImpactRemover(0.5),1.696166,1.333806,1.604167,1.996439,1.516562,1.629428
DisparateImpactRemover(0.7),1.812287,1.417569,1.737473,1.829023,1.504714,1.660213


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.064132,0.027666,0.056061,0.093348,0.070543,0.062350
AdversarialDebiase,0.008282,0.052339,0.045152,0.022147,0.035906,0.032765
GridSearchReduction(EO),0.037746,0.016216,0.042525,0.052625,0.036219,0.037066
GridSearchReduction(DP),0.037009,0.014088,0.036263,0.046883,0.036432,0.034135
Reweight,0.068592,0.030854,0.060303,0.081929,0.055906,0.059517
DisparateImpactRemover(0.5),0.062102,0.034043,0.061515,0.085515,0.059370,0.060509
DisparateImpactRemover(0.7),0.068838,0.041008,0.068384,0.074241,0.058465,0.062187


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.130262,-0.102078,0.034698,0.125424,-0.037054,0.030250
AdversarialDebiase,0.021585,-0.000779,0.055314,-0.031162,-0.030675,0.002856
GridSearchReduction(EO),0.078340,-0.142078,0.009033,0.023341,-0.061900,-0.018653
GridSearchReduction(DP),0.062822,-0.132381,-0.000990,0.009153,-0.053932,-0.023066
Reweight,0.156143,-0.084416,0.041528,0.092809,-0.060742,0.029064
DisparateImpactRemover(0.5),0.135417,-0.096104,0.051106,0.121380,-0.043138,0.033732
DisparateImpactRemover(0.7),0.125054,-0.071775,0.073578,0.090242,-0.043138,0.034792


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.024258,0.003925,0.009230,0.036828,0.044597,0.023768
AdversarialDebiase,-0.017635,0.011934,0.000151,-0.003841,0.002899,-0.001298
GridSearchReduction(EO),0.003201,-0.002984,-0.001334,0.013252,0.007557,0.003938
GridSearchReduction(DP),0.006047,-0.003909,-0.006877,0.010238,0.004896,0.002079
Reweight,0.023108,0.004541,0.014089,0.031378,0.031734,0.020970
DisparateImpactRemover(0.5),0.020262,0.007225,0.010835,0.029933,0.033063,0.020264
DisparateImpactRemover(0.7),0.031100,0.011142,0.014371,0.025143,0.031955,0.022742


In [15]:
#Feldman et al., 2015 implementation
def DisparateImpactRemover_10_CV(train_set,test_set):
    #Training
    di = DisparateImpactRemover(repair_level=1.0,sensitive_attribute="gender")
    train_di = di.fit_transform(train_set.copy(deepcopy=True))
    test_di = di.fit_transform(test_set.copy(deepcopy=True))

    di_model=RandomForestClassifier()
    di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)
    #Testing
    pred=di_model.predict(test_di.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("DisparateImpactRemover(1.0)",DisparateImpactRemover_10_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.8565,0.8665,0.874,0.8575,0.8565,0.8622
AdversarialDebiase,0.8475,0.8680,0.854,0.8540,0.8490,0.8545
GridSearchReduction(EO),0.8605,0.8675,0.872,0.8580,0.8570,0.8630
GridSearchReduction(DP),0.8625,0.8640,0.873,0.8550,0.8580,0.8625
Reweight,0.8600,0.8640,0.870,0.8570,0.8570,0.8616
DisparateImpactRemover(0.5),0.8575,0.8725,0.873,0.8550,0.8555,0.8627
DisparateImpactRemover(0.7),0.8590,0.8710,0.875,0.8515,0.8550,0.8623
DisparateImpactRemover(1.0),0.8590,0.8710,0.877,0.8585,0.8590,0.8649


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.712446,0.747826,0.812245,0.740458,0.784983,0.759592
AdversarialDebiase,0.668050,0.762332,0.760369,0.828729,0.790076,0.761911
GridSearchReduction(EO),0.729614,0.741667,0.796813,0.763485,0.795775,0.765471
GridSearchReduction(DP),0.744493,0.746606,0.803213,0.746939,0.797203,0.767691
Reweight,0.732456,0.733906,0.793522,0.743191,0.781879,0.756991
DisparateImpactRemover(0.5),0.716738,0.766949,0.791506,0.741036,0.791519,0.761550
DisparateImpactRemover(0.7),0.726087,0.761702,0.813765,0.729839,0.788732,0.764025
DisparateImpactRemover(1.0),0.718487,0.766234,0.824490,0.758065,0.796552,0.772765


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.430052,0.451444,0.491358,0.472019,0.506608,0.470296
AdversarialDebiase,0.417098,0.446194,0.407407,0.364964,0.455947,0.418322
GridSearchReduction(EO),0.440415,0.467192,0.493827,0.447689,0.497797,0.469384
GridSearchReduction(DP),0.437824,0.433071,0.493827,0.445255,0.502203,0.462436
Reweight,0.432642,0.448819,0.483951,0.464720,0.513216,0.468670
DisparateImpactRemover(0.5),0.432642,0.475066,0.506173,0.452555,0.493392,0.471966
DisparateImpactRemover(0.7),0.432642,0.469816,0.496296,0.440389,0.493392,0.466507
DisparateImpactRemover(1.0),0.443005,0.464567,0.498765,0.457421,0.508811,0.474514


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.726181,1.271284,1.576324,2.064563,1.613777,1.650426
AdversarialDebiase,1.070873,1.602470,1.512027,1.276063,1.312409,1.354768
GridSearchReduction(EO),1.377793,1.144313,1.399810,1.547707,1.287931,1.351511
GridSearchReduction(DP),1.380614,1.135629,1.335201,1.465358,1.287550,1.320870
Reweight,1.817999,1.302545,1.625786,1.905439,1.450924,1.620539
DisparateImpactRemover(0.5),1.696166,1.333806,1.604167,1.996439,1.516562,1.629428
DisparateImpactRemover(0.7),1.812287,1.417569,1.737473,1.829023,1.504714,1.660213
DisparateImpactRemover(1.0),1.615396,1.448533,1.629630,1.798258,1.498529,1.598069


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.064132,0.027666,0.056061,0.093348,0.070543,0.062350
AdversarialDebiase,0.008282,0.052339,0.045152,0.022147,0.035906,0.032765
GridSearchReduction(EO),0.037746,0.016216,0.042525,0.052625,0.036219,0.037066
GridSearchReduction(DP),0.037009,0.014088,0.036263,0.046883,0.036432,0.034135
Reweight,0.068592,0.030854,0.060303,0.081929,0.055906,0.059517
DisparateImpactRemover(0.5),0.062102,0.034043,0.061515,0.085515,0.059370,0.060509
DisparateImpactRemover(0.7),0.068838,0.041008,0.068384,0.074241,0.058465,0.062187
DisparateImpactRemover(1.0),0.057642,0.042778,0.060101,0.072230,0.059102,0.058371


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.130262,-0.102078,0.034698,0.125424,-0.037054,0.030250
AdversarialDebiase,0.021585,-0.000779,0.055314,-0.031162,-0.030675,0.002856
GridSearchReduction(EO),0.078340,-0.142078,0.009033,0.023341,-0.061900,-0.018653
GridSearchReduction(DP),0.062822,-0.132381,-0.000990,0.009153,-0.053932,-0.023066
Reweight,0.156143,-0.084416,0.041528,0.092809,-0.060742,0.029064
DisparateImpactRemover(0.5),0.135417,-0.096104,0.051106,0.121380,-0.043138,0.033732
DisparateImpactRemover(0.7),0.125054,-0.071775,0.073578,0.090242,-0.043138,0.034792
DisparateImpactRemover(1.0),0.083494,-0.058442,0.047914,0.070145,-0.059800,0.016662


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.024258,0.003925,0.009230,0.036828,0.044597,0.023768
AdversarialDebiase,-0.017635,0.011934,0.000151,-0.003841,0.002899,-0.001298
GridSearchReduction(EO),0.003201,-0.002984,-0.001334,0.013252,0.007557,0.003938
GridSearchReduction(DP),0.006047,-0.003909,-0.006877,0.010238,0.004896,0.002079
Reweight,0.023108,0.004541,0.014089,0.031378,0.031734,0.020970
DisparateImpactRemover(0.5),0.020262,0.007225,0.010835,0.029933,0.033063,0.020264
DisparateImpactRemover(0.7),0.031100,0.011142,0.014371,0.025143,0.031955,0.022742
DisparateImpactRemover(1.0),0.026794,0.010834,0.009913,0.025267,0.035725,0.021707


In [16]:
#Pleiss et al., 2017 implementation
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing

def CalibratedEqOddsPostprocessing_CV(train_set,test_set):
    #Training
    ceop_model=RandomForestClassifier()
    ceop_model.fit(train_set.features,train_set.labels.ravel())

    train_eop_set=train_set.copy(deepcopy=True)
    train_eop_set.scores=ceop_model.predict(train_set.features)

    ceop=CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups)
    ceop.fit(train_set,train_eop_set)
    #Testing
    pred=ceop_model.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.scores=pred
    test_pred=ceop.predict(test_pred)
    return test_pred

cross_validation("CalibratedEqOddsPostprocessing",CalibratedEqOddsPostprocessing_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.8565,0.8665,0.874,0.8575,0.8565,0.8622
AdversarialDebiase,0.8475,0.8680,0.854,0.8540,0.8490,0.8545
GridSearchReduction(EO),0.8605,0.8675,0.872,0.8580,0.8570,0.8630
GridSearchReduction(DP),0.8625,0.8640,0.873,0.8550,0.8580,0.8625
Reweight,0.8600,0.8640,0.870,0.8570,0.8570,0.8616
DisparateImpactRemover(0.5),0.8575,0.8725,0.873,0.8550,0.8555,0.8627
DisparateImpactRemover(0.7),0.8590,0.8710,0.875,0.8515,0.8550,0.8623
DisparateImpactRemover(1.0),0.8590,0.8710,0.877,0.8585,0.8590,0.8649
CalibratedEqOddsPostprocessing,0.8600,0.8720,0.876,0.8545,0.8570,0.8639


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.712446,0.747826,0.812245,0.740458,0.784983,0.759592
AdversarialDebiase,0.668050,0.762332,0.760369,0.828729,0.790076,0.761911
GridSearchReduction(EO),0.729614,0.741667,0.796813,0.763485,0.795775,0.765471
GridSearchReduction(DP),0.744493,0.746606,0.803213,0.746939,0.797203,0.767691
Reweight,0.732456,0.733906,0.793522,0.743191,0.781879,0.756991
DisparateImpactRemover(0.5),0.716738,0.766949,0.791506,0.741036,0.791519,0.761550
DisparateImpactRemover(0.7),0.726087,0.761702,0.813765,0.729839,0.788732,0.764025
DisparateImpactRemover(1.0),0.718487,0.766234,0.824490,0.758065,0.796552,0.772765
CalibratedEqOddsPostprocessing,0.730435,0.759336,0.815261,0.736220,0.787671,0.765785


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.430052,0.451444,0.491358,0.472019,0.506608,0.470296
AdversarialDebiase,0.417098,0.446194,0.407407,0.364964,0.455947,0.418322
GridSearchReduction(EO),0.440415,0.467192,0.493827,0.447689,0.497797,0.469384
GridSearchReduction(DP),0.437824,0.433071,0.493827,0.445255,0.502203,0.462436
Reweight,0.432642,0.448819,0.483951,0.464720,0.513216,0.468670
DisparateImpactRemover(0.5),0.432642,0.475066,0.506173,0.452555,0.493392,0.471966
DisparateImpactRemover(0.7),0.432642,0.469816,0.496296,0.440389,0.493392,0.466507
DisparateImpactRemover(1.0),0.443005,0.464567,0.498765,0.457421,0.508811,0.474514
CalibratedEqOddsPostprocessing,0.435233,0.480315,0.501235,0.454988,0.506608,0.475676


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.726181,1.271284,1.576324,2.064563,1.613777,1.650426
AdversarialDebiase,1.070873,1.602470,1.512027,1.276063,1.312409,1.354768
GridSearchReduction(EO),1.377793,1.144313,1.399810,1.547707,1.287931,1.351511
GridSearchReduction(DP),1.380614,1.135629,1.335201,1.465358,1.287550,1.320870
Reweight,1.817999,1.302545,1.625786,1.905439,1.450924,1.620539
DisparateImpactRemover(0.5),1.696166,1.333806,1.604167,1.996439,1.516562,1.629428
DisparateImpactRemover(0.7),1.812287,1.417569,1.737473,1.829023,1.504714,1.660213
DisparateImpactRemover(1.0),1.615396,1.448533,1.629630,1.798258,1.498529,1.598069
CalibratedEqOddsPostprocessing,1.687535,1.457659,1.544444,1.901221,1.496530,1.617478


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.064132,0.027666,0.056061,0.093348,0.070543,0.062350
AdversarialDebiase,0.008282,0.052339,0.045152,0.022147,0.035906,0.032765
GridSearchReduction(EO),0.037746,0.016216,0.042525,0.052625,0.036219,0.037066
GridSearchReduction(DP),0.037009,0.014088,0.036263,0.046883,0.036432,0.034135
Reweight,0.068592,0.030854,0.060303,0.081929,0.055906,0.059517
DisparateImpactRemover(0.5),0.062102,0.034043,0.061515,0.085515,0.059370,0.060509
DisparateImpactRemover(0.7),0.068838,0.041008,0.068384,0.074241,0.058465,0.062187
DisparateImpactRemover(1.0),0.057642,0.042778,0.060101,0.072230,0.059102,0.058371
CalibratedEqOddsPostprocessing,0.060719,0.045377,0.054444,0.080706,0.059314,0.060112


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.130262,-0.102078,0.034698,0.125424,-0.037054,0.030250
AdversarialDebiase,0.021585,-0.000779,0.055314,-0.031162,-0.030675,0.002856
GridSearchReduction(EO),0.078340,-0.142078,0.009033,0.023341,-0.061900,-0.018653
GridSearchReduction(DP),0.062822,-0.132381,-0.000990,0.009153,-0.053932,-0.023066
Reweight,0.156143,-0.084416,0.041528,0.092809,-0.060742,0.029064
DisparateImpactRemover(0.5),0.135417,-0.096104,0.051106,0.121380,-0.043138,0.033732
DisparateImpactRemover(0.7),0.125054,-0.071775,0.073578,0.090242,-0.043138,0.034792
DisparateImpactRemover(1.0),0.083494,-0.058442,0.047914,0.070145,-0.059800,0.016662
CalibratedEqOddsPostprocessing,0.109482,-0.032468,0.042296,0.105714,-0.019233,0.041158


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.024258,0.003925,0.009230,0.036828,0.044597,0.023768
AdversarialDebiase,-0.017635,0.011934,0.000151,-0.003841,0.002899,-0.001298
GridSearchReduction(EO),0.003201,-0.002984,-0.001334,0.013252,0.007557,0.003938
GridSearchReduction(DP),0.006047,-0.003909,-0.006877,0.010238,0.004896,0.002079
Reweight,0.023108,0.004541,0.014089,0.031378,0.031734,0.020970
DisparateImpactRemover(0.5),0.020262,0.007225,0.010835,0.029933,0.033063,0.020264
DisparateImpactRemover(0.7),0.031100,0.011142,0.014371,0.025143,0.031955,0.022742
DisparateImpactRemover(1.0),0.026794,0.010834,0.009913,0.025267,0.035725,0.021707
CalibratedEqOddsPostprocessing,0.024568,0.006433,0.004089,0.027703,0.024414,0.017441


In [17]:
#Kamiran et al., 2012 implementation
from aif360.algorithms.postprocessing import RejectOptionClassification

def RejectOptionClassification_EO_CV(train_set,test_set):
    #Training
    roc_EO_model=RandomForestClassifier()
    roc_EO_model.fit(train_set.features,train_set.labels.ravel())

    train_roc_EO_set=train_set.copy(deepcopy=True)
    train_roc_EO_set.scores=roc_EO_model.predict(train_set.features)

    roc_EO = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    metric_name="Equal opportunity difference")
    roc_EO.fit(train_set, train_roc_EO_set)
    #Testing
    pred=roc_EO_model.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.scores=pred
    test_pred=roc_EO.predict(test_pred)
    return test_pred

cross_validation("RejectOptionClassification(EO)",RejectOptionClassification_EO_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.8565,0.8665,0.8740,0.8575,0.8565,0.8622
AdversarialDebiase,0.8475,0.8680,0.8540,0.8540,0.8490,0.8545
GridSearchReduction(EO),0.8605,0.8675,0.8720,0.8580,0.8570,0.8630
GridSearchReduction(DP),0.8625,0.8640,0.8730,0.8550,0.8580,0.8625
Reweight,0.8600,0.8640,0.8700,0.8570,0.8570,0.8616
DisparateImpactRemover(0.5),0.8575,0.8725,0.8730,0.8550,0.8555,0.8627
DisparateImpactRemover(0.7),0.8590,0.8710,0.8750,0.8515,0.8550,0.8623
DisparateImpactRemover(1.0),0.8590,0.8710,0.8770,0.8585,0.8590,0.8649
CalibratedEqOddsPostprocessing,0.8600,0.8720,0.8760,0.8545,0.8570,0.8639
RejectOptionClassification(EO),0.8570,0.8625,0.8695,0.8530,0.8530,0.8590


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.712446,0.747826,0.812245,0.740458,0.784983,0.759592
AdversarialDebiase,0.668050,0.762332,0.760369,0.828729,0.790076,0.761911
GridSearchReduction(EO),0.729614,0.741667,0.796813,0.763485,0.795775,0.765471
GridSearchReduction(DP),0.744493,0.746606,0.803213,0.746939,0.797203,0.767691
Reweight,0.732456,0.733906,0.793522,0.743191,0.781879,0.756991
DisparateImpactRemover(0.5),0.716738,0.766949,0.791506,0.741036,0.791519,0.761550
DisparateImpactRemover(0.7),0.726087,0.761702,0.813765,0.729839,0.788732,0.764025
DisparateImpactRemover(1.0),0.718487,0.766234,0.824490,0.758065,0.796552,0.772765
CalibratedEqOddsPostprocessing,0.730435,0.759336,0.815261,0.736220,0.787671,0.765785
RejectOptionClassification(EO),0.715517,0.726496,0.788000,0.731225,0.773973,0.747042


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.430052,0.451444,0.491358,0.472019,0.506608,0.470296
AdversarialDebiase,0.417098,0.446194,0.407407,0.364964,0.455947,0.418322
GridSearchReduction(EO),0.440415,0.467192,0.493827,0.447689,0.497797,0.469384
GridSearchReduction(DP),0.437824,0.433071,0.493827,0.445255,0.502203,0.462436
Reweight,0.432642,0.448819,0.483951,0.464720,0.513216,0.468670
DisparateImpactRemover(0.5),0.432642,0.475066,0.506173,0.452555,0.493392,0.471966
DisparateImpactRemover(0.7),0.432642,0.469816,0.496296,0.440389,0.493392,0.466507
DisparateImpactRemover(1.0),0.443005,0.464567,0.498765,0.457421,0.508811,0.474514
CalibratedEqOddsPostprocessing,0.435233,0.480315,0.501235,0.454988,0.506608,0.475676
RejectOptionClassification(EO),0.430052,0.446194,0.486420,0.450122,0.497797,0.462117


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.726181,1.271284,1.576324,2.064563,1.613777,1.650426
AdversarialDebiase,1.070873,1.602470,1.512027,1.276063,1.312409,1.354768
GridSearchReduction(EO),1.377793,1.144313,1.399810,1.547707,1.287931,1.351511
GridSearchReduction(DP),1.380614,1.135629,1.335201,1.465358,1.287550,1.320870
Reweight,1.817999,1.302545,1.625786,1.905439,1.450924,1.620539
DisparateImpactRemover(0.5),1.696166,1.333806,1.604167,1.996439,1.516562,1.629428
DisparateImpactRemover(0.7),1.812287,1.417569,1.737473,1.829023,1.504714,1.660213
DisparateImpactRemover(1.0),1.615396,1.448533,1.629630,1.798258,1.498529,1.598069
CalibratedEqOddsPostprocessing,1.687535,1.457659,1.544444,1.901221,1.496530,1.617478
RejectOptionClassification(EO),1.713299,1.406748,1.687831,1.921234,1.538547,1.653532


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.064132,0.027666,0.056061,0.093348,0.070543,0.062350
AdversarialDebiase,0.008282,0.052339,0.045152,0.022147,0.035906,0.032765
GridSearchReduction(EO),0.037746,0.016216,0.042525,0.052625,0.036219,0.037066
GridSearchReduction(DP),0.037009,0.014088,0.036263,0.046883,0.036432,0.034135
Reweight,0.068592,0.030854,0.060303,0.081929,0.055906,0.059517
DisparateImpactRemover(0.5),0.062102,0.034043,0.061515,0.085515,0.059370,0.060509
DisparateImpactRemover(0.7),0.068838,0.041008,0.068384,0.074241,0.058465,0.062187
DisparateImpactRemover(1.0),0.057642,0.042778,0.060101,0.072230,0.059102,0.058371
CalibratedEqOddsPostprocessing,0.060719,0.045377,0.054444,0.080706,0.059314,0.060112
RejectOptionClassification(EO),0.062994,0.039945,0.065657,0.081639,0.063359,0.062719


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.130262,-0.102078,0.034698,0.125424,-0.037054,0.030250
AdversarialDebiase,0.021585,-0.000779,0.055314,-0.031162,-0.030675,0.002856
GridSearchReduction(EO),0.078340,-0.142078,0.009033,0.023341,-0.061900,-0.018653
GridSearchReduction(DP),0.062822,-0.132381,-0.000990,0.009153,-0.053932,-0.023066
Reweight,0.156143,-0.084416,0.041528,0.092809,-0.060742,0.029064
DisparateImpactRemover(0.5),0.135417,-0.096104,0.051106,0.121380,-0.043138,0.033732
DisparateImpactRemover(0.7),0.125054,-0.071775,0.073578,0.090242,-0.043138,0.034792
DisparateImpactRemover(1.0),0.083494,-0.058442,0.047914,0.070145,-0.059800,0.016662
CalibratedEqOddsPostprocessing,0.109482,-0.032468,0.042296,0.105714,-0.019233,0.041158
RejectOptionClassification(EO),0.119899,-0.066753,0.065980,0.087288,-0.052990,0.030685


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.024258,0.003925,0.009230,0.036828,0.044597,0.023768
AdversarialDebiase,-0.017635,0.011934,0.000151,-0.003841,0.002899,-0.001298
GridSearchReduction(EO),0.003201,-0.002984,-0.001334,0.013252,0.007557,0.003938
GridSearchReduction(DP),0.006047,-0.003909,-0.006877,0.010238,0.004896,0.002079
Reweight,0.023108,0.004541,0.014089,0.031378,0.031734,0.020970
DisparateImpactRemover(0.5),0.020262,0.007225,0.010835,0.029933,0.033063,0.020264
DisparateImpactRemover(0.7),0.031100,0.011142,0.014371,0.025143,0.031955,0.022742
DisparateImpactRemover(1.0),0.026794,0.010834,0.009913,0.025267,0.035725,0.021707
CalibratedEqOddsPostprocessing,0.024568,0.006433,0.004089,0.027703,0.024414,0.017441
RejectOptionClassification(EO),0.025335,0.012375,0.014490,0.034268,0.041271,0.025548


In [18]:
#Kamiran et al., 2012 implementation
from aif360.algorithms.postprocessing import RejectOptionClassification

def RejectOptionClassification_DP_CV(train_set,test_set):
    #Training
    roc_EO_model=RandomForestClassifier()
    roc_EO_model.fit(train_set.features,train_set.labels.ravel())

    train_roc_EO_set=train_set.copy(deepcopy=True)
    train_roc_EO_set.scores=roc_EO_model.predict(train_set.features)

    roc_EO = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    metric_name="Statistical parity difference")
    roc_EO.fit(train_set, train_roc_EO_set)
    #Testing
    pred=roc_EO_model.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.scores=pred
    test_pred=roc_EO.predict(test_pred)
    return test_pred

cross_validation("RejectOptionClassification(DP)",RejectOptionClassification_DP_CV)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
c:\Users\15783\s

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.8565,0.8665,0.8740,0.8575,0.8565,0.8622
AdversarialDebiase,0.8475,0.8680,0.8540,0.8540,0.8490,0.8545
GridSearchReduction(EO),0.8605,0.8675,0.8720,0.8580,0.8570,0.8630
GridSearchReduction(DP),0.8625,0.8640,0.8730,0.8550,0.8580,0.8625
Reweight,0.8600,0.8640,0.8700,0.8570,0.8570,0.8616
DisparateImpactRemover(0.5),0.8575,0.8725,0.8730,0.8550,0.8555,0.8627
DisparateImpactRemover(0.7),0.8590,0.8710,0.8750,0.8515,0.8550,0.8623
DisparateImpactRemover(1.0),0.8590,0.8710,0.8770,0.8585,0.8590,0.8649
CalibratedEqOddsPostprocessing,0.8600,0.8720,0.8760,0.8545,0.8570,0.8639
RejectOptionClassification(EO),0.8570,0.8625,0.8695,0.8530,0.8530,0.8590


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.712446,0.747826,0.812245,0.740458,0.784983,0.759592
AdversarialDebiase,0.668050,0.762332,0.760369,0.828729,0.790076,0.761911
GridSearchReduction(EO),0.729614,0.741667,0.796813,0.763485,0.795775,0.765471
GridSearchReduction(DP),0.744493,0.746606,0.803213,0.746939,0.797203,0.767691
Reweight,0.732456,0.733906,0.793522,0.743191,0.781879,0.756991
DisparateImpactRemover(0.5),0.716738,0.766949,0.791506,0.741036,0.791519,0.761550
DisparateImpactRemover(0.7),0.726087,0.761702,0.813765,0.729839,0.788732,0.764025
DisparateImpactRemover(1.0),0.718487,0.766234,0.824490,0.758065,0.796552,0.772765
CalibratedEqOddsPostprocessing,0.730435,0.759336,0.815261,0.736220,0.787671,0.765785
RejectOptionClassification(EO),0.715517,0.726496,0.788000,0.731225,0.773973,0.747042


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.430052,0.451444,0.491358,0.472019,0.506608,0.470296
AdversarialDebiase,0.417098,0.446194,0.407407,0.364964,0.455947,0.418322
GridSearchReduction(EO),0.440415,0.467192,0.493827,0.447689,0.497797,0.469384
GridSearchReduction(DP),0.437824,0.433071,0.493827,0.445255,0.502203,0.462436
Reweight,0.432642,0.448819,0.483951,0.464720,0.513216,0.468670
DisparateImpactRemover(0.5),0.432642,0.475066,0.506173,0.452555,0.493392,0.471966
DisparateImpactRemover(0.7),0.432642,0.469816,0.496296,0.440389,0.493392,0.466507
DisparateImpactRemover(1.0),0.443005,0.464567,0.498765,0.457421,0.508811,0.474514
CalibratedEqOddsPostprocessing,0.435233,0.480315,0.501235,0.454988,0.506608,0.475676
RejectOptionClassification(EO),0.430052,0.446194,0.486420,0.450122,0.497797,0.462117


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,1.726181,1.271284,1.576324,2.064563,1.613777,1.650426
AdversarialDebiase,1.070873,1.602470,1.512027,1.276063,1.312409,1.354768
GridSearchReduction(EO),1.377793,1.144313,1.399810,1.547707,1.287931,1.351511
GridSearchReduction(DP),1.380614,1.135629,1.335201,1.465358,1.287550,1.320870
Reweight,1.817999,1.302545,1.625786,1.905439,1.450924,1.620539
DisparateImpactRemover(0.5),1.696166,1.333806,1.604167,1.996439,1.516562,1.629428
DisparateImpactRemover(0.7),1.812287,1.417569,1.737473,1.829023,1.504714,1.660213
DisparateImpactRemover(1.0),1.615396,1.448533,1.629630,1.798258,1.498529,1.598069
CalibratedEqOddsPostprocessing,1.687535,1.457659,1.544444,1.901221,1.496530,1.617478
RejectOptionClassification(EO),1.713299,1.406748,1.687831,1.921234,1.538547,1.653532


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.064132,0.027666,0.056061,0.093348,0.070543,0.062350
AdversarialDebiase,0.008282,0.052339,0.045152,0.022147,0.035906,0.032765
GridSearchReduction(EO),0.037746,0.016216,0.042525,0.052625,0.036219,0.037066
GridSearchReduction(DP),0.037009,0.014088,0.036263,0.046883,0.036432,0.034135
Reweight,0.068592,0.030854,0.060303,0.081929,0.055906,0.059517
DisparateImpactRemover(0.5),0.062102,0.034043,0.061515,0.085515,0.059370,0.060509
DisparateImpactRemover(0.7),0.068838,0.041008,0.068384,0.074241,0.058465,0.062187
DisparateImpactRemover(1.0),0.057642,0.042778,0.060101,0.072230,0.059102,0.058371
CalibratedEqOddsPostprocessing,0.060719,0.045377,0.054444,0.080706,0.059314,0.060112
RejectOptionClassification(EO),0.062994,0.039945,0.065657,0.081639,0.063359,0.062719


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.130262,-0.102078,0.034698,0.125424,-0.037054,0.030250
AdversarialDebiase,0.021585,-0.000779,0.055314,-0.031162,-0.030675,0.002856
GridSearchReduction(EO),0.078340,-0.142078,0.009033,0.023341,-0.061900,-0.018653
GridSearchReduction(DP),0.062822,-0.132381,-0.000990,0.009153,-0.053932,-0.023066
Reweight,0.156143,-0.084416,0.041528,0.092809,-0.060742,0.029064
DisparateImpactRemover(0.5),0.135417,-0.096104,0.051106,0.121380,-0.043138,0.033732
DisparateImpactRemover(0.7),0.125054,-0.071775,0.073578,0.090242,-0.043138,0.034792
DisparateImpactRemover(1.0),0.083494,-0.058442,0.047914,0.070145,-0.059800,0.016662
CalibratedEqOddsPostprocessing,0.109482,-0.032468,0.042296,0.105714,-0.019233,0.041158
RejectOptionClassification(EO),0.119899,-0.066753,0.065980,0.087288,-0.052990,0.030685


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.024258,0.003925,0.009230,0.036828,0.044597,0.023768
AdversarialDebiase,-0.017635,0.011934,0.000151,-0.003841,0.002899,-0.001298
GridSearchReduction(EO),0.003201,-0.002984,-0.001334,0.013252,0.007557,0.003938
GridSearchReduction(DP),0.006047,-0.003909,-0.006877,0.010238,0.004896,0.002079
Reweight,0.023108,0.004541,0.014089,0.031378,0.031734,0.020970
DisparateImpactRemover(0.5),0.020262,0.007225,0.010835,0.029933,0.033063,0.020264
DisparateImpactRemover(0.7),0.031100,0.011142,0.014371,0.025143,0.031955,0.022742
DisparateImpactRemover(1.0),0.026794,0.010834,0.009913,0.025267,0.035725,0.021707
CalibratedEqOddsPostprocessing,0.024568,0.006433,0.004089,0.027703,0.024414,0.017441
RejectOptionClassification(EO),0.025335,0.012375,0.014490,0.034268,0.041271,0.025548


In [22]:
Accuracy_data.to_csv("bank_accuracy.csv")
Precision_data.to_csv("bank_precision.csv")
Recall_data.to_csv("bank_recall.csv")
DisparateImpact_data.to_csv("bank_disparate_impact.csv")
DisparateParity_data.to_csv("bank_disparate_parity.csv")
EqualOpportunity_data.to_csv("bank_equal_opportunity.csv")
EqualizedOdds_data.to_csv("bank_equalized_odds.csv")